In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import numpy as np
import warnings

warnings.filterwarnings('ignore', category = FutureWarning)

In [4]:
path_scrapper = "/content/drive/MyDrive/ProjetML/Base_BookCrossing/Base_completee/"
file_scrapper = path_scrapper + "bothWebSites_InternetSearch_AllBooks_BookCrossing.csv"
file_scrapper_cleaned = path_scrapper + "bothWebSites_InternetSearch_AllBooks_BookCrossing_cleaned.csv"

In [ ]:
books_scrapped = pd.read_csv(file_scrapper, sep = ';', parse_dates=['Year-Of-Publication'])
len(books_scrapped)

114528

In [ ]:
books_scrapped_cleaned = books_scrapped.copy()

# Transformation of column names to be as our SQL data base
Columns :
    - 'book_id'
    - 'average_rating'
    - 'issue_flag'
    - 'is_active' 
    - 'data_source' 
do not exist in books_scrapped

In [ ]:
books_scrapped_cleaned.rename( columns = {'ISBN_10': 'isbn', 'ISBN_13': 'isbn_13', 'Book-Title': 'book_title', \
                                  'Book-Author': 'book_author', 'Year-Of-Publication': 'year_of_publication', \
                                  'Description': 'book_description', 'Language': 'book_language', \
                                  'Pages': 'number_of_pages', 'Awards': 'awards', 'Author\'s genre': 'author_genres', \
                                  'Same serie': 'books_in_series'}, inplace = True )

In [ ]:
books_scrapped_cleaned.to_csv(file_scrapper_cleaned, sep = "|", index = False)

# Creation of a second column for the second author gender

In [ ]:
books_scrapped_cleaned = pd.read_csv(file_scrapper_cleaned, sep = '|')

In [ ]:
books_scrapped_cleaned.head(20)

,isbn,isbn_13,OtherID,book_title,book_author,year_of_publication,Publisher,Category,book_description,book_language,Image,number_of_pages,awards,author_genres,books_in_series
0,195153448,"9,78E+12",0.0,Classical Mythology,Mark P. O. Morford,2003-01-01,"Oxford University Press, USA",Social Science,Provides an introduction to classical myths pl...,en,http://books.google.com/books/content?id=YABYK...,808.0,NaN,NaN,NaN
1,2005018,"9,78E+12",0.0,Clara Callan : A Novel,Richard Bruce Wright,2001-01-01,HarperFlamingo Canada,Actresses,"In a small town in Canada, Clara Callan reluct...",en,http://books.google.com/books/content?id=yfx0v...,414.0,"\nScotiabank Giller Prize (2001), Governor Gen...",NaN,[]
2,60973129,0,IND:30000026059836,Decision in Normandy,Carlo D'Este,1991-01-01,Harper Perennial,1940-1949,"Here, for the first time in paperback, is an o...",en,http://books.google.com/books/content?id=_LufA...,555.0,NaN,History,[]
3,374157065,"9,78E+12",0,Flu : The Story of the Great Influenza Pandemi...,Gina Bari Kolata,1999-01-01,Macmillan,Medical,"Describes the great flu epidemic of 1918, an o...",en,http://books.google.com/books/content?id=GkthX...,330.0,NaN,Health,[]
4,393045218,"9,78E+12",0.0,The Mummies of Ürümchi,E. J. W. Barber,1999-01-01,W. W. Norton,Design,A look at the incredibly well-preserved ancien...,en,http://books.google.com/books/content?id=5OujQ...,240.0,\nKiriyama Prize Nominee for Nonfiction (1999)\n,History,[]
5,399135782,"9,78E+12",0.0,The Kitchen God's Wife,Amy Tan,1991-01-01,Putnam Publishing Group,Fiction,A Chinese immigrant who is convinced she is dy...,en,http://books.google.com/books/content?id=s6IRK...,415.0,NaN,Literature & Fiction,[]
6,425176428,"9,78E+12",0.0,What If? : The World's Foremost Military Histo...,Robert Cowley,2000-01-01,Berkley Publishing Group,History,"Essays by respected military historians, inclu...",en,http://books.google.com/books/content?id=LxevD...,395.0,NaN,History,"['https://www.goodreads.com/book/show/406281',..."
7,671870432,"9,78E+12",0.0,Pleading Guilty,Scott Turow,1993-01-01,Simon & Schuster Audio,Fiction,NaN,en,NaN,0.0,NaN,Mystery & Thrillers,"['https://www.goodreads.com/book/show/425029',..."
8,679425608,0,UVA:35007001566771,Under the Black Flag : The Romance and the Rea...,David Cordingly,1995-01-01,Random House Incorporated,Fiction,"An authoritative, revisionist history of the g...",en,http://books.google.com/books/content?id=slQSA...,296.0,NaN,History,[]
9,074322678X,0.0,PSU:000049152678,Where You'll Find Me : And Other Stories,Ann Beattie,2002-01-01,Scribner,Fiction,"Now back in print, Ann Beattie&#39;s finest sh...",un,http://books.google.com/books/content?id=twdaA...,207.0,NaN,Literature & Fiction,[]


In [ ]:
#new column created
books_scrapped_cleaned.insert(14, "author_genres_other", pd.Series())

#We only deal with not empty cells
aut_gen = books_scrapped_cleaned.author_genres[books_scrapped_cleaned["author_genres"].isna() == False]

for i in aut_gen.index:
    gender = books_scrapped_cleaned["author_genres"][i].strip()

    res_split = gender.strip().split('&')

    #In case a split has been done:
    ##### the second author gender must be copied in author_genres_other column
    ##### and the first author gender only must appear in author_genres column
    if(len(res_split) > 1):
        books_scrapped_cleaned["author_genres_other"][i] = res_split[1].strip()
        books_scrapped_cleaned["author_genres"][i] = res_split[0].strip()       

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value i

In [ ]:
books_scrapped_cleaned.head(20)

,isbn,isbn_13,OtherID,book_title,book_author,year_of_publication,Publisher,Category,book_description,book_language,Image,number_of_pages,awards,author_genres,author_genres_other,books_in_series
0,195153448,"9,78E+12",0.0,Classical Mythology,Mark P. O. Morford,2003-01-01,"Oxford University Press, USA",Social Science,Provides an introduction to classical myths pl...,en,http://books.google.com/books/content?id=YABYK...,808.0,NaN,NaN,NaN,NaN
1,2005018,"9,78E+12",0.0,Clara Callan : A Novel,Richard Bruce Wright,2001-01-01,HarperFlamingo Canada,Actresses,"In a small town in Canada, Clara Callan reluct...",en,http://books.google.com/books/content?id=yfx0v...,414.0,"\nScotiabank Giller Prize (2001), Governor Gen...",NaN,NaN,[]
2,60973129,0,IND:30000026059836,Decision in Normandy,Carlo D'Este,1991-01-01,Harper Perennial,1940-1949,"Here, for the first time in paperback, is an o...",en,http://books.google.com/books/content?id=_LufA...,555.0,NaN,History,NaN,[]
3,374157065,"9,78E+12",0,Flu : The Story of the Great Influenza Pandemi...,Gina Bari Kolata,1999-01-01,Macmillan,Medical,"Describes the great flu epidemic of 1918, an o...",en,http://books.google.com/books/content?id=GkthX...,330.0,NaN,Health,NaN,[]
4,393045218,"9,78E+12",0.0,The Mummies of Ürümchi,E. J. W. Barber,1999-01-01,W. W. Norton,Design,A look at the incredibly well-preserved ancien...,en,http://books.google.com/books/content?id=5OujQ...,240.0,\nKiriyama Prize Nominee for Nonfiction (1999)\n,History,NaN,[]
5,399135782,"9,78E+12",0.0,The Kitchen God's Wife,Amy Tan,1991-01-01,Putnam Publishing Group,Fiction,A Chinese immigrant who is convinced she is dy...,en,http://books.google.com/books/content?id=s6IRK...,415.0,NaN,Literature,Fiction,[]
6,425176428,"9,78E+12",0.0,What If? : The World's Foremost Military Histo...,Robert Cowley,2000-01-01,Berkley Publishing Group,History,"Essays by respected military historians, inclu...",en,http://books.google.com/books/content?id=LxevD...,395.0,NaN,History,NaN,"['https://www.goodreads.com/book/show/406281',..."
7,671870432,"9,78E+12",0.0,Pleading Guilty,Scott Turow,1993-01-01,Simon & Schuster Audio,Fiction,NaN,en,NaN,0.0,NaN,Mystery,Thrillers,"['https://www.goodreads.com/book/show/425029',..."
8,679425608,0,UVA:35007001566771,Under the Black Flag : The Romance and the Rea...,David Cordingly,1995-01-01,Random House Incorporated,Fiction,"An authoritative, revisionist history of the g...",en,http://books.google.com/books/content?id=slQSA...,296.0,NaN,History,NaN,[]
9,074322678X,0.0,PSU:000049152678,Where You'll Find Me : And Other Stories,Ann Beattie,2002-01-01,Scribner,Fiction,"Now back in print, Ann Beattie&#39;s finest sh...",un,http://books.google.com/books/content?id=twdaA...,207.0,NaN,Literature,Fiction,[]


In [ ]:
books_scrapped_cleaned.to_csv(file_scrapper_cleaned, sep = "|", index = False)

# Cleaning of awards column

In [ ]:
books_scrapped_cleaned = pd.read_csv(file_scrapper_cleaned, sep = '|')

## Analysis of awards

In [ ]:
#We only deal with not empty cells
awards_lines = books_scrapped_cleaned.author_genres[books_scrapped_cleaned["awards"].isna() == False]

#study of the number of awards inside each cell
#study of the kind of awards
nb_awards = []
awards_name = []

for i in awards_lines.index:

    #first we retrieve the "\n", " " ...
    awards_cleaned = books_scrapped_cleaned["awards"][i].strip()

    #then we split when there are several awards in the same cell
    awards_split = awards_cleaned.split(',')
    l = len(awards_split)

    #27 awards for those books !
    if l == 27:
      print("\n\ntitle {0}, at indice {1}\n".format(books_scrapped_cleaned["book_title"][i], i))
      
    nb_awards.append(l)

    #In case a split has been done:
    if(l > 1):
        for j in range(l):
          #we keep only the name, not the year of obtention
          temp = awards_split[j].strip()
          awards_name.append(temp.split("(")[0].strip())
      



title Harry Potter and the Sorcerer's Stone, at indice 2143



title Harry Potter and the Sorcerer's Stone, at indice 2809



title Harry Potter and the Sorcerer's Stone, at indice 9027



title Harry Potter y la piedra filosofal, at indice 21110



title Harry Potter and the Sorcerer's Stone, at indice 28428



title Harry Potter und der Stein der Weisen, at indice 34539



title Harrius Potter et Philosophi Lapis : (Harry Potter and the Philosopher's Stone), at indice 35740



title Harry Potter and the Philosopher's Stone, at indice 50436



title Harry Potter e la pietra filosofale : romanzo, at indice 51993



title Harry Potter and the Sorcerer's Stone, at indice 52581



title Harry Potter and the Sorcerer's Stone, at indice 55787



title Harry Potter à l'école des sorciers, at indice 64658



title Harry Potter and the Sorcerer's Stone, at indice 72188



title Harry Potter and the Sorcerer's Stone, at indice 77387



title Harry Potter and the Philosopher's Stone : Irish Ga

In [ ]:
books_scrapped_cleaned["awards"][2809]

"\nMythopoeic Fantasy Award for Children's Literature (2008), British Book Award for Children's Book of the Year (1998), Prijs van de Nederlandse Kinderjury for 6-9 jaar en 10-12 jaar (2002), American Booksellers Book Of The Year  Award for Children (1999), Audie Award (2000)\n...more\nWest Australian Young Readers' Book Award (WAYRBA) for Younger Readers (2000), South Carolina Book Award for Junior Book Award (2001), Grand Canyon Reader Award for Teen Book (2000), Charlotte Award (2000), Nene Award (2000), Massachusetts Children's Book Award (2000), Colorado Blue Spruce Young Adult Book Award (2001), Blue Hen Book Award for Chapter Book (2001), Nevada Young Readers' Award for Young Reader Category (2000), Golden Archer Award for Middle/Junior High (2000), Indian Paintbrush Book Award (2000), Hotze de Roosprijs (2002), Nestlé Smarties Book Prize for 9–11 years (1997), Eliot Rosewater Indiana High School Book Award (2001), Kinderboekwinkelprijs (1999), Parenting Book of the Year Award (

In [ ]:
pd.Series(nb_awards).value_counts()

1     8858
2     3077
3     1249
4      658
5      641
6      143
7       83
8       69
10      28
12      27
9       19
15      16
21      16
27      16
11      11
16       7
14       5
20       4
19       2
13       2
17       1
dtype: int64

In [ ]:
max(nb_awards)

27

In [ ]:
stats_awards = pd.Series(awards_name).value_counts()
stats_awards

Dorothy Canfield Fisher Children's Book Award Nominee                       428
National Book Award Finalist for Fiction                                    233
National Book Critics Circle Award Nominee for Fiction                      224
Hugo Award Nominee for Best Novel                                           210
Newbery Medal Nominee                                                       202
                                                                           ... 
Wilhelm-Hauff-Preis                                                           1
Lambda Literary Award for Gay Men's Science Fiction/Fantasy                   1
Seiun Award 星雲賞 for Best Translated Short Story for "Story of Your Life"      1
Analog Award Nominee for Best Serial Novel or Novella for "Anasazi"           1
Gordon Montador Award                                                         1
Length: 2033, dtype: int64

In [ ]:
stats_awards[stats_awards.values > 100]

Dorothy Canfield Fisher Children's Book Award Nominee     428
National Book Award Finalist for Fiction                  233
National Book Critics Circle Award Nominee for Fiction    224
Hugo Award Nominee for Best Novel                         210
Newbery Medal Nominee                                     202
Newbery Medal                                             173
Nebula Award Nominee for Best Novel                       173
Los Angeles Times Book Prize Nominee for Fiction          171
Margaret A. Edwards Award                                 166
Hugo Award for Best Novel                                 161
Locus Award Nominee for Best Fantasy Novel                159
World Fantasy Award Nominee for Best Novel                156
Booker Prize Nominee                                      154
Lewis Carroll Shelf Award                                 144
Books I Loved Best Yearly                                 144
Pulitzer Prize for Fiction                                142
Internat

In [ ]:
type(stats_awards)

pandas.core.series.Series

In [ ]:
stats_awards.to_csv(path_scrapper + "awards_names.csv", sep = "|", index = True)

## Cleaning of awards

In [ ]:
stats_awards[0]

428

In [ ]:
stats_awards.index[0]

"Dorothy Canfield Fisher Children's Book Award Nominee"

In [ ]:
def searchInFamousAwards(list_awards):
  print("\n\n")

  l_entry = len(list_awards)
  l_ref = len(stats_awards)
  awards_returned = ""

  i = 0
  #among all awards existing, we search for the most famous appearing in list_awards
  while((awards_returned == "") and (i < l_ref)):
    if stats_awards.index[i] in list_awards:
      awards_returned = stats_awards.index[i] 
    i = i + 1  

  if(awards_returned == ""):
    awards_returned = list_awards[0]

  return  awards_returned


In [ ]:
books_scrapped_cleaned = pd.read_csv(file_scrapper_cleaned, sep = '|')

In [ ]:
#new column created
books_scrapped_cleaned["awards_cleaned"] = pd.Series()

#We only deal with not empty cells
awards_lines = books_scrapped_cleaned.author_genres[books_scrapped_cleaned["awards"].isna() == False]

for i in awards_lines.index:
    awards_cleaned = []

    #first we retrieve the "\n", " " ...
    awards_cleaned = books_scrapped_cleaned["awards"][i].strip()

    #then we split when there are several awards in the same cell
    awards_split = awards_cleaned.split(',')
    l = len(awards_split)

    #first we clean all the aards (because split introduce " ")
    for j in range(l):
      temp = awards_split[j].strip()
      awards_split[j] = temp.split("(")[0].strip()

    #In case a split has been done:
    if(l > 1):
        res_awards = searchInFamousAwards(awards_split)
        if res_awards != "":
          books_scrapped_cleaned["awards_cleaned"][i] = res_awards
        else:
          books_scrapped_cleaned["awards_cleaned"][i] = awards_split[0]



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.




























































































































































































































































































































































































































































































































































































































































































































































































































































































































































In [ ]:
books_scrapped_cleaned["awards_cleaned"]

0                         NaN
1         Trillium Book Award
2                         NaN
3                         NaN
4                         NaN
                 ...         
114523                    NaN
114524                    NaN
114525                    NaN
114526                    NaN
114527                    NaN
Name: awards_cleaned, Length: 114528, dtype: object

In [ ]:
books_scrapped_cleaned["awards_cleaned"].value_counts()

Dorothy Canfield Fisher Children's Book Award Nominee                428
National Book Award Finalist for Fiction                             233
Hugo Award Nominee for Best Novel                                    210
National Book Critics Circle Award Nominee for Fiction               156
Books I Loved Best Yearly                                            138
                                                                    ... 
Analog Award Nominee for Best Novella/Novelette for "The Locusts"      1
Prijs van de Nederlandse Kinderjury for 6-9 jaar                       1
Will Eisner Comic Industry Awards for Best Comics-Related Book         1
Compton Crook Award                                                    1
O. Henry Award                                                         1
Name: awards_cleaned, Length: 613, dtype: int64

In [ ]:
del books_scrapped_cleaned["awards"]

In [ ]:
books_scrapped_cleaned.rename(columns = {"awards_cleaned": "awards"}, inplace = True)

In [ ]:
books_scrapped_cleaned.head()

,isbn,isbn_13,OtherID,book_title,book_author,year_of_publication,Publisher,Category,book_description,book_language,Image,number_of_pages,author_genres,author_genres_other,books_in_series,awards
0,195153448,"9,78E+12",0.0,Classical Mythology,Mark P. O. Morford,2003-01-01,"Oxford University Press, USA",Social Science,Provides an introduction to classical myths pl...,en,http://books.google.com/books/content?id=YABYK...,808.0,NaN,NaN,NaN,NaN
1,2005018,"9,78E+12",0.0,Clara Callan : A Novel,Richard Bruce Wright,2001-01-01,HarperFlamingo Canada,Actresses,"In a small town in Canada, Clara Callan reluct...",en,http://books.google.com/books/content?id=yfx0v...,414.0,NaN,NaN,[],Trillium Book Award
2,60973129,0,IND:30000026059836,Decision in Normandy,Carlo D'Este,1991-01-01,Harper Perennial,1940-1949,"Here, for the first time in paperback, is an o...",en,http://books.google.com/books/content?id=_LufA...,555.0,History,NaN,[],NaN
3,374157065,"9,78E+12",0,Flu : The Story of the Great Influenza Pandemi...,Gina Bari Kolata,1999-01-01,Macmillan,Medical,"Describes the great flu epidemic of 1918, an o...",en,http://books.google.com/books/content?id=GkthX...,330.0,Health,NaN,[],NaN
4,393045218,"9,78E+12",0.0,The Mummies of Ürümchi,E. J. W. Barber,1999-01-01,W. W. Norton,Design,A look at the incredibly well-preserved ancien...,en,http://books.google.com/books/content?id=5OujQ...,240.0,History,NaN,[],NaN


In [ ]:
books_scrapped_cleaned.to_csv(file_scrapper_cleaned, sep = "|", index = False)

# Création of a second column for Category

## First we separate categories with "&" in two columns
The column "Category_other" has been created to take the second part of such categories (part after "&")

In [22]:
books_scrapped_cleaned = pd.read_csv(file_scrapper_cleaned, sep = '|')

In [6]:
books_scrapped_cleaned.head()

,isbn,isbn_13,OtherID,book_title,book_author,year_of_publication,Publisher,Category,book_description,book_language,Image,number_of_pages,author_genres,author_genres_other,books_in_series,awards
0,195153448,"9,78E+12",0.0,Classical Mythology,Mark P. O. Morford,2003-01-01,"Oxford University Press, USA",Social Science,Provides an introduction to classical myths pl...,en,http://books.google.com/books/content?id=YABYK...,808.0,NaN,NaN,NaN,NaN
1,2005018,"9,78E+12",0.0,Clara Callan : A Novel,Richard Bruce Wright,2001-01-01,HarperFlamingo Canada,Actresses,"In a small town in Canada, Clara Callan reluct...",en,http://books.google.com/books/content?id=yfx0v...,414.0,NaN,NaN,[],Trillium Book Award
2,60973129,0,IND:30000026059836,Decision in Normandy,Carlo D'Este,1991-01-01,Harper Perennial,1940-1949,"Here, for the first time in paperback, is an o...",en,http://books.google.com/books/content?id=_LufA...,555.0,History,NaN,[],NaN
3,374157065,"9,78E+12",0,Flu : The Story of the Great Influenza Pandemi...,Gina Bari Kolata,1999-01-01,Macmillan,Medical,"Describes the great flu epidemic of 1918, an o...",en,http://books.google.com/books/content?id=GkthX...,330.0,Health,NaN,[],NaN
4,393045218,"9,78E+12",0.0,The Mummies of Ürümchi,E. J. W. Barber,1999-01-01,W. W. Norton,Design,A look at the incredibly well-preserved ancien...,en,http://books.google.com/books/content?id=5OujQ...,240.0,History,NaN,[],NaN


In [19]:
#new column created
books_scrapped_cleaned.insert(8, "Category_other", pd.Series())

#We only deal with not empty cells
cate_gen = books_scrapped_cleaned.author_genres[books_scrapped_cleaned["Category"].isna() == False]

for i in cate_gen.index:
    cate = books_scrapped_cleaned["Category"][i].strip()
    res_split = cate.split('&')

    #In case a split has been done:
    ##### the second category must be copied in Category_other column
    ##### and the first category only must appear in Category column
    if(len(res_split) > 1):
        books_scrapped_cleaned["Category_other"][i] = res_split[1].strip()
        books_scrapped_cleaned["Category"][i] = res_split[0].strip()       

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarnin

In [20]:
books_scrapped_cleaned.head()

,isbn,isbn_13,OtherID,book_title,book_author,year_of_publication,Publisher,Category,Category_other,book_description,book_language,Image,number_of_pages,author_genres,author_genres_other,books_in_series,awards
0,195153448,"9,78E+12",0.0,Classical Mythology,Mark P. O. Morford,2003-01-01,"Oxford University Press, USA",Social Science,NaN,Provides an introduction to classical myths pl...,en,http://books.google.com/books/content?id=YABYK...,808.0,NaN,NaN,NaN,NaN
1,2005018,"9,78E+12",0.0,Clara Callan : A Novel,Richard Bruce Wright,2001-01-01,HarperFlamingo Canada,Actresses,NaN,"In a small town in Canada, Clara Callan reluct...",en,http://books.google.com/books/content?id=yfx0v...,414.0,NaN,NaN,[],Trillium Book Award
2,60973129,0,IND:30000026059836,Decision in Normandy,Carlo D'Este,1991-01-01,Harper Perennial,1940-1949,NaN,"Here, for the first time in paperback, is an o...",en,http://books.google.com/books/content?id=_LufA...,555.0,History,NaN,[],NaN
3,374157065,"9,78E+12",0,Flu : The Story of the Great Influenza Pandemi...,Gina Bari Kolata,1999-01-01,Macmillan,Medical,NaN,"Describes the great flu epidemic of 1918, an o...",en,http://books.google.com/books/content?id=GkthX...,330.0,Health,NaN,[],NaN
4,393045218,"9,78E+12",0.0,The Mummies of Ürümchi,E. J. W. Barber,1999-01-01,W. W. Norton,Design,NaN,A look at the incredibly well-preserved ancien...,en,http://books.google.com/books/content?id=5OujQ...,240.0,History,NaN,[],NaN


In [15]:
 books_scrapped_cleaned["Category_other"].value_counts()

Autobiography               3072
Economics                    983
Spirit                       905
Relationships                880
Fitness                      843
Disciplines                  372
Recreation                   349
Graphic Novels               317
Hobbies                      293
Home                         150
Engineering                  123
Activities                   114
Collectibles                  74
AUTOBIOGRAPHY                  9
GRAPHIC NOVELS.                3
RELATIONSHIPS                  3
ECONOMICS                      2
Answers                        1
SPIRIT                         1
FITNESS                        1
mystery stories, English       1
GRAPHIC NOVELS                 1
Name: Category_other, dtype: int64

In [17]:
 books_scrapped_cleaned["Category"].head(150)

0          Social Science
1               Actresses
2               1940-1949
3                 Medical
4                  Design
              ...        
145               Fiction
146            Philosophy
147    Literary Criticism
148               Fiction
149              Classics
Name: Category, Length: 150, dtype: object

## The same for categories with "(...)" (in Category column)
The part between "(...)" has been put in the second column

In [28]:
#We only deal with not empty cells
cate_gen = books_scrapped_cleaned.author_genres[books_scrapped_cleaned["Category"].isna() == False]

for i in cate_gen.index:
    cate = books_scrapped_cleaned["Category"][i].strip()
    res_split_ent = cate.split('(')

    if (len(res_split_ent) > 1):
      res_split_out = res_split_ent[1].split(')')

      #In case a split has been done:
      ##### the second term (between "(...)") must be copied in Category_other column
      ##### and the first term remains in Category column (without the second term)
      books_scrapped_cleaned["Category"][i] = res_split_ent[0].strip()
      books_scrapped_cleaned["Category_other"][i] = res_split_out[0].strip()
    

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [29]:
books_scrapped_cleaned["Category"].head(50)

0       Social Science
1            Actresses
2            1940-1949
3              Medical
4               Design
5              Fiction
6              History
7              Fiction
8              Fiction
9              Fiction
10             Fiction
11             History
12             Fiction
13                 NaN
14          Nonfiction
15             Mystery
16              Nature
17             Fiction
18             Fiction
19             Fiction
20               Humor
21             Cooking
22           Reference
23    Juvenile Fiction
24    Juvenile Fiction
25    Juvenile Fiction
26             Fiction
27             Fiction
28             Fiction
29             Fiction
30             Fiction
31             Fiction
32    Canadian fiction
33                 NaN
34          Historical
35                 NaN
36             Fiction
37             Fiction
38             History
39             Fiction
40             Fiction
41            Classics
42              Health
43         

## Then removing of "," 
THe second term is, as previously, put inside "Category_other" column

In [34]:
#We only deal with not empty cells
cate_gen = books_scrapped_cleaned.author_genres[books_scrapped_cleaned["Category"].isna() == False]

for i in cate_gen.index:
    cate = books_scrapped_cleaned["Category"][i].strip()
    res_split = cate.split(',')
    second_term = ""

    if (len(res_split) > 1):
      #In case a split has been done:
      ##### and the first term (before ",") remains in Category column (without the second term)
      books_scrapped_cleaned["Category"][i] = res_split[0].strip()

      ##### all the terms appearing after "," must be copied in Category_other column
      for j in range(1,len(res_split)):
        second_term = second_term + res_split[j]
      books_scrapped_cleaned["Category_other"][i] = second_term.strip()

    

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [35]:
books_scrapped_cleaned["Category"].value_counts()

Fiction                    40402
Juvenile Fiction            5893
Biography                   3341
Mystery                     2894
History                     1998
                           ...  
Trans-Alaska Pipeline          1
Feral children                 1
Deutsch - Sprachkritik         1
Bereavement in children        1
Knots and splices              1
Name: Category, Length: 4579, dtype: int64

In [36]:
books_scrapped_cleaned.to_csv(file_scrapper_cleaned, sep = "|", index = False)